# Explore Discharge Predictions (CamelsGB)

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
from pathlib import Path
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

base_dir = Path(".").absolute().parents[0]
import sys

if sys.path[0] != base_dir.as_posix():
    sys.path = [base_dir.as_posix()] + sys.path

In [ ]:
from spatio_temporal.config import Config
from spatio_temporal.training.trainer import Trainer
from spatio_temporal.training.tester import Tester
from tests.utils import (
    create_linear_ds,
    _test_sklearn_model,
    download_test_jena_data,
    load_test_jena_data_as_dataset
)
from spatio_temporal.training.eval_utils import _plot_loss_curves, save_losses

# Load in config file and predictions

In [ ]:
ds = xr.open_dataset(base_dir / "data/ALL_dynamic_ds.nc")
ds

In [ ]:
run_dir = base_dir / "runs/runoff_step10/"
cfg = Config(run_dir / "config.yml")
cfg

In [ ]:
xr_path = sorted(list(run_dir.glob("*.nc")))[-1]
preds = xr.open_dataset(xr_path).isel(horizon=0).drop("horizon")
preds

In [ ]:
for i in range(5):
    f, ax = plt.subplots(figsize=(12, 4))
    pixel = np.random.choice(preds.pixel.values)
    preds.sel(time="2007", pixel=pixel).to_dataframe().plot(ax=ax)

# Test specific epoch

In [ ]:
# # sorted(list(run_dir.glob("*.pt")))
# cfg._cfg["run_dir"] = run_dir
# cfg._cfg["device"] = "cpu"
# cfg._cfg["num_workers"] = 4
# tester = Tester(cfg, ds)

In [ ]:
# tester.run_test(epoch=5)